# migration digraph

In [11]:
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
import networkx as nx 

from fonction_rtet import troncons_non_mailles, troncons_peu_mailles, gr_maillage

rtet_path = '../rtet/'
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}
vl_central_2025 = (True, 400, 150, 60)
vl_central_2027 = (True, 600, 150, 60)
vl_global_2027 = (False, 300, 150, 60)
pl_global_2025 = (False, 1400, 350, 120)
pl_central_2027 = (True, 2800, 350, 120)


version = 'V1'
scenario = vl_global_2027

## Chargement AFIR



In [12]:
central, pcum, pmax, dist = scenario
add_name = '_core' if central else ''
add_pcum = '_' + str(pcum)    
add_pmax = '_' + str(pmax)

gr_tot = gnx.from_geopandas_edgelist(gpd.read_file(rtet_path + version + add_name + add_pcum + add_pmax + '_afir_edge.geojson'), 
                                     node_gdf=gpd.read_file(rtet_path + version + add_name + add_pcum + add_pmax + '_afir_node.geojson'),
                                     node_attr=True, edge_attr=True)
#gr_tot.clean_attributes()

In [13]:
len(gr_tot.nodes), len(gr_tot.edges)

(3713, 3786)

In [14]:
dgr_tot = gr_tot.to_directed()

In [15]:
stat_edges = [edge for edge in dgr_tot.edges if dgr_tot.edges[edge]['nature'][:7] == 'liaison']
dgs = nx.edge_subgraph(dgr_tot, stat_edges)
rtet_edges = [edge for edge in dgr_tot.edges if dgr_tot.edges[edge]['nature'][:7] != 'liaison']
dgr = nx.edge_subgraph(dgr_tot, rtet_edges)

### Trajet entre deux stations
Exemple entre la station '4128' (Bollene) et la station '4148' (saint-martin de crau)

In [16]:
bollene = [node for node in dgr_tot.nodes if dgr_tot.nodes[node].get('id_station') == 'FR*IZF*EFAST*95*1*3']
bollene = bollene[0] if bollene else None
st_martin_de_crau = [node for node in dgr_tot.nodes if dgr_tot.nodes[node].get('id_station') == 'FR*PVD*EVG*SMC13*D01*1']
st_martin_de_crau = st_martin_de_crau[0] if st_martin_de_crau else None
trajet = bollene and st_martin_de_crau
print('trajet : ', bollene, st_martin_de_crau)

trajet :  27965 28271


In [18]:
if trajet:
    print('distance entre les stations : ',
    nx.shortest_path_length(dgr_tot, source=bollene, target=st_martin_de_crau, weight='weight'))

distance entre les stations :  116138.54862765616


In [19]:
if trajet:
    path = nx.shortest_path(dgr_tot, source=bollene, target=st_martin_de_crau, weight='weight')
    gr_path = nx.subgraph_view(dgr_tot, filter_edge=(lambda x, y: x in path and y in path and abs(path.index(y) - path.index(x)) == 1), filter_node=lambda x: x in path)
    print(path, type(path), nx.path_weight(gr_tot, path, 'weight'))

[27965, 804, 566, 567, 2026, 250, 794, 568, 785, 564, 775, 767, 760, 309, 744, 748, 182, 747, 743, 28271] <class 'list'> 116138.54862765616


In [20]:
param_exp_gs = {'e_tooltip': ["source", "target"], 'e_popup': ['nature', 'weight', 'source', 'target'], 'e_color': 'grey',
                'n_name': 'station', 'e_name': 'liaison station', 'n_popup': ['amenageur', 'operateur', 'p_cum', 'p_max', 'node_id', 'id_station'], 
                'n_tooltip': "amenageur", 'n_color': 'green', 'n_marker_kwds': {'radius': 4, 'fill': True}}

param_exp_gr = {'e_popup': ['weight', 'source', 'target', 'core', "CORRIDORS", "INTROADNUM", "NATIONALRO", "ID", "nature"], 
                'n_popup': ['roadname', 'nom', 'node_id'],
                'e_tooltip': ["source", "target", "nature", "core"], 
                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
#param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 'e_popup': ['weight', 'source', 'target'], 'e_tooltip': ["source", "target"], 
#                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
param_exp_path = {'e_name': 'path', 'e_color': 'red'}
refmap = {'tiles': 'cartodbpositron', 'location': [43.8, 5], 'zoom_start': 9}

if trajet:    
    carte = dgs.explore(refmap=refmap, **param_exp_gs)
    carte = dgr.explore(refmap=carte, n_color='black', e_name='edges rtet', n_name='nodes rtet', **param_exp_gr)
    carte = dgr_path.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

AttributeError: 'DiGraph' object has no attribute 'explore'